In [1]:
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
import pandas as pd
import seaborn as sns

from sklearn.metrics import confusion_matrix, f1_score
from sklearn.metrics import mean_squared_error, median_absolute_error
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet, ElasticNetCV

from joblib import dump, load

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [2]:
conn_dict = {'host': 'localhost',
        'dbname': 'insight',
        'user': os.getenv('PG_USER'),
        'password': os.getenv('PG_PASSWORD')}

In [3]:
with psycopg2.connect(**conn_dict) as conn, conn.cursor() as cur:
    cur.execute('SELECT * FROM patterns3')
    res = cur.fetchall()
    cols = [desc[0] for desc in cur.description]

In [4]:
DF = pd.DataFrame(res, columns=cols)


# EDA and Feature Eng.

In [5]:
df = DF.copy()

In [6]:
df.loc[:, 'price'] = df.price.apply(lambda x: float(x) if pd.notnull(x) else 0)

In [7]:
df.loc[:, 'pubyear'] = df.published.apply(lambda x: x.year if pd.notnull(x) else None)

In [8]:
df.loc[:, 'buttoned_mod'] = df[['attribute_buttoned', 'attribute_buttonholes']].max(axis=1)

### Features that appear frequently enough in patterns

In [9]:
attribute_cols = [a for a in df.columns if a.startswith('attribute_')]
att_df = df[attribute_cols].sum(axis=0).reset_index().sort_values(0, ascending=False)
att_df.columns = ['attribute', 'pattern_count']
att_df.loc[:, 'pattern_share'] = att_df.pattern_count / df.shape[0]
# att_df[att_df.pattern_share >= 0.05]

### Needles that appear frequently

In [10]:
needles_cols = [n for n in df.columns if n.startswith('needles_')]
needle_df = df[needles_cols].sum(axis=0).reset_index().sort_values(0, ascending=False)
needle_df.columns = ['needle_size', 'pattern_count']
needle_df.loc[:, 'pattern_share'] = needle_df.pattern_count / df.shape[0]
# needle_df[needle_df.patte rn_share >= 0.01]

In [11]:

numeric_cols = ['difficulty_average','num_photos']

cat_cols = ['pattern_type','yarn_weight']


attribute_cols = ['attribute_adult', #
                  'attribute_female',#
                  'attribute_worked_flat',#
                  'attribute_worked_in_the_round',#
                  'attribute_chart',#
                  'attribute_bottom_up',#
                  'attribute_one_piece',#
                  'attribute_unisex',#
                  'attribute_seamed',#
                  'attribute_lace',#
                  'attribute_teen',#
                  'attribute_ribbed_ribbing',#
                  'attribute_textured',#
                  'attribute_cables',#
                  'attribute_stripes_colorwork',#
                  'attribute_top_down',#
                  'attribute_child',#
                  'attribute_long',#
                  'attribute_stranded',#
                  'attribute_baby',#
                  'attribute_positive_ease',#
                  'attribute_has_schematic',#
                  'attribute_male',#
                  'attribute_eyelets',#
                  'attribute_toddler', #
                  'attribute_fitted',#
                  'attribute_short_rows',
                  'buttoned_mod']

needles_cols = ['needles_us_6',
                'needles_us_7',
                'needles_us_4',
                'needles_us_8',
                'needles_us_5',
                'needles_us_3',
                'needles_us_2h',
                'needles_us_10',
                'needles_us_9',
                'needles_us_1h',
                'needles_us_2',
                'needles_us_1',
                'needles_us_11',
                'needles_us_10h',
                'needles_us_13',
                'needles_us_15',
                'needles_us_0']

bool_cols = needles_cols + attribute_cols

for cat_col in cat_cols:
    df.loc[:, cat_col] = df[cat_col].apply(lambda x: str(x))

for num_col in numeric_cols:
    df.loc[:, num_col] = df[num_col].apply(lambda x: float(x))

In [12]:
df = df[['price'] + bool_cols + numeric_cols + cat_cols].dropna()

## Training data

In [29]:
df.price.head()

0    0.0
1    4.9
2    6.0
3    6.0
4    0.0
Name: price, dtype: float64

In [13]:
X = df[bool_cols + numeric_cols + cat_cols]
y = df.price.apply(lambda x: int(x > 0))  # is item for sale (1) or free (0)

print(f'Data is {X.shape[0]} rows and {X.shape[1]} cols')
print(f'{np.round(100*np.mean(y), 2)}% of patterns are free')

Data is 393467 rows and 49 cols
48.7% of patterns are free


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [15]:
print(f'Training data shape: {X_train.shape}')

Training data shape: (314773, 49)


### 1. Elastic Net Logistic to predict free or not

In [18]:
cats = [list(set(X.pattern_type)),list(set(X.yarn_weight))]

preprocess = make_column_transformer(
    (StandardScaler(), numeric_cols + bool_cols),
    (OneHotEncoder(categories=cats), cat_cols))

sgd_params = dict(validation_fraction=0.1,
                  penalty='elasticnet',
                  loss='log',
                  random_state=42,
                  max_iter=10)

pipeline = make_pipeline(preprocess, SGDClassifier(**sgd_params))

%time pipeline.fit(X_train, y_train)
print("ElasticNet SGDClassifier regression score: %f" % pipeline.score(X_test, y_test))

/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


CPU times: user 9.26 s, sys: 364 ms, total: 9.62 s
Wall time: 5.24 s
ElasticNet SGDClassifier regression score: 0.701934


/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


#### Get feature names and coefficients of the best model

In [19]:
standardscaler_features = preprocess.transformers_[0][2]
ohe_features = [cat_cols[i] + '_' + item for i, l in enumerate(preprocess.transformers_[1][1].categories) for item in l]
ftrs = standardscaler_features + ohe_features

coefs = pipeline.named_steps['sgdclassifier'].coef_[0]
model_results = pd.DataFrame({'feature': ftrs, 'coef': coefs}).sort_values('coef', ascending=True)

In [20]:
model_results[model_results.feature != 0].tail()

,feature,coef
57,pattern_type_mittens,0.357186
80,yarn_weight_Cobweb,0.423011
78,yarn_weight_Lace,0.447861
77,yarn_weight_Fingering,0.512317
1,num_photos,1.070976


In [21]:
print(f"Model F1 score: {f1_score(y_test, pipeline.predict(X_test))}")

Model F1 score: 0.6756098910217404


/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


## Models to predict price for sale items

In [24]:
df2 = DF[(DF.currency == 'USD') & \
             (DF.price <= 17) & \
             (DF.price > 0) & \
             pd.notnull(DF.price)]

df2.loc[:, 'buttoned_mod'] = df2[['attribute_buttoned', 'attribute_buttonholes']].max(axis=1)
df2.loc[:, 'price'] = df2.price.apply(lambda x: float(x) if pd.notnull(x) else 0)
df2.loc[:, 'projects_count'] = df2.projects_count.apply(lambda x: float(x) if pd.notnull(x) else 0)
for cat_col in cat_cols:
    df2.loc[:, cat_col] = df2[cat_col].apply(lambda x: str(x))

for num_col in numeric_cols + ['projects_count']:
    df2.loc[:, num_col] = df2[num_col].apply(lambda x: float(x))

/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [25]:
'price' in df2.columns

True

In [26]:
df2 = df2[['price']+ bool_cols + numeric_cols + cat_cols].dropna()
y2 = df2.price
X2 = df2[bool_cols + numeric_cols + cat_cols]
X2_train, X2_test, y2_train, y2_test = \
train_test_split(X2, y2, test_size=0.2, random_state=42)

### Random Forest Regression

In [27]:
cats = [list(set(X2.pattern_type)),list(set(X2.yarn_weight))]

preprocess = make_column_transformer(
    (StandardScaler(), numeric_cols + bool_cols),
    (OneHotEncoder(categories=cats), cat_cols))

rf_pipe = make_pipeline(preprocess, RandomForestRegressor(random_state=42, n_estimators=100))
%time rf_pipe.fit(X2_train, y2_train)

/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


CPU times: user 1min 31s, sys: 311 ms, total: 1min 32s
Wall time: 1min 30s


Pipeline(memory=None,
     steps=[('columntransformer', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True), ['difficulty_average', 'num_photos', 'needles_us_6', 'needles_us_...mators=100, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [33]:
from sklearn.metrics import median_absolute_error

print(f"MSE Test: {np.sqrt(mean_squared_error(y2_test, rf_pipe.predict(X2_test)))}")
print(f"MAE Test: {median_absolute_error(y2_test, rf_pipe.predict(X2_test))}")

/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


MSE Test: 1.430337306505398


/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


MAE Test: 0.7504000000000008


In [29]:
print(f"Standard dev. of number of priojects: {np.std(df2.price)}")

Standard dev. of number of priojects: 1.626368014606781


## Benchmark: category means and medians

In [31]:
z = df2.groupby('pattern_type').agg({'price': 'median'}).reset_index()
median_map = dict(zip(z.pattern_type, z.price))
X2_test.loc[:, 'cat_mean'] = X2.pattern_type.apply(lambda x: median_map[x])

/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [32]:
print(f"Median Abs error: {median_absolute_error(y2_test, X2_test.cat_mean)}")
print(f"MSE: {np.sqrt(mean_squared_error(y2_test, X2_test.cat_mean))}")

Median Abs error: 1.0
MSE: 1.5172880794354668


In [37]:
len(rf_pipe.predict(X2_test))

/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


26380

In [41]:
df2.loc[:, 'pred_price'] = rf_pipe.predict(X2)

/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


In [44]:
df2.drop('price_pred', axis=1, inplace=True)

In [54]:
df2 = DF[(DF.currency == 'USD') & \
             (DF.price <= 17) & \
             (DF.price > 0) & \
             pd.notnull(DF.price)]

df2.loc[:, 'buttoned_mod'] = df2[['attribute_buttoned', 'attribute_buttonholes']].max(axis=1)
df2.loc[:, 'price'] = df2.price.apply(lambda x: float(x) if pd.notnull(x) else 0)
df2.loc[:, 'projects_count'] = df2.projects_count.apply(lambda x: float(x) if pd.notnull(x) else 0)
for cat_col in cat_cols:
    df2.loc[:, cat_col] = df2[cat_col].apply(lambda x: str(x))

for num_col in numeric_cols + ['projects_count']:
    df2.loc[:, num_col] = df2[num_col].apply(lambda x: float(x))

df2 = df2[['projects_count']+ bool_cols + numeric_cols + cat_cols].dropna()
X2 = df2[bool_cols + numeric_cols + cat_cols]
X2.loc[:, 'pred_price'] = rf_pipe.predict(X2)
y2 = df2['projects_count']

/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype bool, float64 were all converted to float6

In [53]:
X2_train, X2_test, y2_train, y2_test = \
train_test_split(X2, y2, test_size=0.2, random_state=42)


,needles_us_6,needles_us_7,needles_us_4,needles_us_8,needles_us_5,needles_us_3,needles_us_2h,needles_us_10,needles_us_9,needles_us_1h,...,attribute_eyelets,attribute_toddler,attribute_fitted,attribute_short_rows,buttoned_mod,difficulty_average,num_photos,pattern_type,yarn_weight,pred_price
393444,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,True,0.0,0.0,vest,Bulky,6.166100
393445,False,False,False,True,False,False,False,True,False,False,...,False,False,False,True,False,0.0,0.0,vest,Worsted,6.201333
393446,False,False,False,False,False,False,False,True,True,False,...,False,False,False,True,True,0.0,0.0,vest,Worsted,5.954467
393447,False,False,False,False,False,False,False,True,True,False,...,False,False,False,False,True,0.0,0.0,vest,Bulky,6.262900
393458,True,False,True,False,False,False,False,False,False,False,...,True,False,False,False,False,0.0,0.0,vest,Light Fingering,4.977500


In [55]:
cats = [list(set(X2.pattern_type)),list(set(X2.yarn_weight))]

preprocess = make_column_transformer(
    (StandardScaler(), numeric_cols + bool_cols),
    (OneHotEncoder(categories=cats), cat_cols))

rf_pipe2 = make_pipeline(preprocess, RandomForestRegressor(random_state=42, n_estimators=100))
%time rf_pipe2.fit(X2_train, y2_train)

/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


CPU times: user 1min 32s, sys: 296 ms, total: 1min 32s
Wall time: 1min 31s


Pipeline(memory=None,
     steps=[('columntransformer', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True), ['difficulty_average', 'num_photos', 'needles_us_6', 'needles_us_...mators=100, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False))])

In [56]:
z = df2.groupby('pattern_type').agg({'projects_count': 'median'}).reset_index()
median_map = dict(zip(z.pattern_type, z.projects_count))
X2_test.loc[:, 'cat_median'] = X2.pattern_type.apply(lambda x: median_map[x])

z = df2.groupby('pattern_type').agg({'projects_count': 'mean'}).reset_index()
mean_map = dict(zip(z.pattern_type, z.projects_count))
X2_test.loc[:, 'cat_mean'] = X2.pattern_type.apply(lambda x: mean_map[x])

/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/pandas/core/indexing.py:480: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [58]:
X2_test.tail()

,needles_us_6,needles_us_7,needles_us_4,needles_us_8,needles_us_5,needles_us_3,needles_us_2h,needles_us_10,needles_us_9,needles_us_1h,...,attribute_toddler,attribute_fitted,attribute_short_rows,buttoned_mod,difficulty_average,num_photos,pattern_type,yarn_weight,cat_mean,cat_median
374483,True,False,True,False,False,False,False,False,False,False,...,True,False,False,True,3.333333,5.0,shrug,DK,18.884532,3.0
52380,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,4.000000,4.0,fingerless,Worsted,18.102145,3.0
10620,True,False,False,False,False,True,False,False,False,False,...,False,False,False,False,4.529412,7.0,shawl-wrap,Fingering,31.493990,6.0
315616,True,False,False,True,True,False,False,False,False,False,...,False,False,False,True,3.500000,3.0,cardigan,Aran,35.650142,4.0
296696,False,False,False,False,False,False,False,False,False,True,...,False,True,False,False,0.000000,1.0,pullover,Sport,48.502337,10.0


In [59]:
print(f'Model MSE: {mean_squared_error(rf_pipe2.predict(X2_test), y2_test)}')
print(f'Mean MSE: {mean_squared_error(X2_test.cat_mean, y2_test)}')
print(f'Median MSE: {mean_squared_error(X2_test.cat_median, y2_test)}')

/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


Model MSE: 2.0458648103811163
Mean MSE: 415.29417450118814
Median MSE: 7.627407240333586


In [60]:
print(f'Model MAE: {median_absolute_error(rf_pipe2.predict(X2_test), y2_test)}')
print(f'Mean MAE: {median_absolute_error(X2_test.cat_mean, y2_test)}')
print(f'Median MAE: {median_absolute_error(X2_test.cat_median, y2_test)}')

/Users/jason/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


Model MAE: 0.7504000000000008
Mean MAE: 14.102144730751426
Median MAE: 2.0
